In [ ]:
rider_stats = spark.sql(
    """
    SELECT rider_id, MONTH(started_at) as month, YEAR(started_at) as year, count(*) as num_rides, AVG( DATEDIFF (minute, started_at, ended_at)) as minutes_spent
    FROM trips
    GROUP by rider_id, MONTH(started_at), YEAR(started_at);
    """
)
rider_stats.createOrReplaceTempView("rider_stats")

In [ ]:
rider_stats_month = spark.sql(
"""
    SELECT rider_id, avg(num_rides) as avg_rides_month,
     avg(minutes_spent) as avg_time_spent_month
    from rider_stats
    GROUP BY rider_id
"""
)
display(rider_stats_month.head())
rider_stats_month.createOrReplaceTempView("rider_stats_month")

Row(rider_id='45273', avg_rides_month=8.090909090909092, avg_time_spent_month=13.65671979535616)

In [ ]:
%sql
insert into dim_rider
(
  select riders.rider_id as id,
  first as firstname,
  last as lastname,
  address,
  birthday,
  account_start_date,
  account_end_date,
  is_member,
  avg_rides_month,
  avg_time_spent_month,
  int (DATEDIFF(year, birthday, account_start_date)) 
  as rider_age_account_start
  from riders 
  join rider_stats_month on (riders.rider_id = rider_stats_month.rider_id)
)

num_affected_rows,num_inserted_rows
37709,37709


In [ ]:
%sql
select * from dim_rider limit 10

id,firstname,lastname,address,birthday,account_start_date,account_end_date,is_member,avg_rides_month,avg_time_spent_month
45273,Robert,Ferguson,1703 Mayer Mission Apt. 363,1983-03-15,2020-12-05,null,true,8,14
60822,Brad,Webb,07339 Powers Highway,1981-08-03,2022-01-24,null,true,17,14
40726,Sara,Mckinney,2417 Maria Isle,1988-06-25,2018-11-18,null,true,6,14
35374,Elizabeth,Taylor,707 Jason Mountain,1996-10-19,2021-12-26,null,true,1,21
52800,Tyler,Myers,996 Alison Island Suite 177,2000-04-28,2021-06-13,2022-01-01,false,5,17
58744,Barbara,Giles,490 Hopkins Falls Suite 366,2006-01-19,2016-12-11,null,true,5,15
57112,Desiree,Miller,7837 Alexander Crossing Apt. 143,2001-10-04,2020-01-07,null,true,22,17
57904,Brandon,Hill,8626 Chandler Mount Apt. 610,2004-02-10,2015-09-09,null,false,26,16
55371,Susan,Brown,89646 Harris Plains,2004-04-04,2019-03-24,null,true,3,16
8304,Robin,Wilson,43332 Larry Cape Apt. 567,1990-09-26,2021-04-21,null,true,1,67
